# About this document

The purpose of this noted version of pipeline is to guide the user through each step of the codes so that they are able to understand them as much as possible and equipped with knowledge to customize the script regardless of programming skills. It is **not** supposed to be runned as production tool. If you click "Run All" on this document, you **will** encounter errors. For productive purpose, consider modifying the accompanying **pipeline.ipynb** to suit your need.

Before we start, it's highly recommended that you get familiar with basic python concepts and operations like [string manipulation](https://docs.python.org/3.4/library/string.html), [tuples, lists and dictionaries](https://docs.python.org/3/tutorial/datastructures.html), as well as a little bit about [object-oriented programming](https://python.swaroopch.com/oop.html) and [python modules](https://docs.python.org/3/tutorial/modules.html).

Another note on the styling of this document: most of the sentences should (hopefully) make sense if taken literally. However, I try to use some syntax highlighting on the texts consistently to demonstrate the close relationship between the codes and thought process, as well as encouraging the reader to understand the already natural and self-explantory Python syntax and jargons. Specifically:

-  a [hyperlink](https://en.wikipedia.org/wiki/Hyperlink) usually point to a well-defined python module or class or methods, especially when that concept is first encountered in this document and the reader would likely need some background reference. The link usually points to the official documentation of that concept, which might not be the best place to start for beginner. If you find the documentation puzzling, try to google that concept and find a tutorial that best suit your preference.
-  an inline `code` usually refer to a name that already exsist in the [namespace](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces) (i.e. the context where we run the codes in this document). It can be a previously encountered concepts, but more often it referes to variable or method names that we [imported](https://docs.python.org/3/reference/import.html) or defined along the way.
-  **bold** texts are used more loosely to highlight anything that requires more attention than plain texts. Though it's not used as carefully as previous formats, it often refer to some specific values that a variable or method arguments can assume.

# Pipeline

## Setting up

The cells under this section should be executed every time the kernel is restarted.

### set module paths and data path

Ideally, the following cell would be the only thing you have to change when analyzing different datasets. `minian_path` should be the path that contains a folder named **"minian"** , under which the actual codes of **minian** (.py files) resides. The default value **\.** means "current folder", which should work in most cases unless you want to try out another specific version of minian that is not in the same folder as this notebook. `dpath` is the folder that contains the actual videos, which are usually named **"msCam\*.avi"** where \* is a number. `meta_dict` is a python [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) that is used to construct meta data for the final labeled data structure. Currently we assume data are stored in some structured folders and we use folder names to fill in information. The default value means: the name of the last folder in `dpath` (the one that directly contains the videos) would be used as the value of a field named **'session_id'**, the name of the second-last folder in `dpath` would be the value for **'session'** and so on. Both the `keys` (field names) and `values` (numbers indicating which level of folder name should be used) of `meta_dict` can be modified to suit the specific user case. `chunks` is used to divide data into smaller chunks for parallel processing. Currently it is rarelly used and has little impact on the actual performance of the script. It is recommended to leave `chunks` to default. `in_memory` specify whether to load the whole dataset into memory. Since out-of-core computation are still not fully supported, it is recommended to leave `in_memory` to `True`.

In [ ]:
minian_path = "."
dpath = "./demo_movies/"
meta_dict={'session_id': -1, 'session': -2, 'animal': -3}
chunks = {'frame': 1000, 'height': 200, 'width': 200, 'unit_id':20}
in_memory = True

### load modules

The following cell loads the necessary modules and usually should not be modified. If you encounter an `ImportError`, check that you followed the installation instructions and that `minian_path` is pointing to the right place.

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(minian_path)
import gc
import psutil
import numpy as np
import xarray as xr
import holoviews as hv
import paramnb
import matplotlib.pyplot as plt
import bokeh.plotting as bpl
import dask.array as da
import pandas as pd
import dask
import datashader as ds
from holoviews.operation.datashader import datashade, regrid, dynspread
from datashader.colors import Sets1to3
from dask.diagnostics import ProgressBar
from IPython.core.display import display, HTML
from dask.distributed import Client, progress
from minian.utilities import load_videos, varray_to_tif, save_cnmf, save_movies, scale_varr, save_variable
from minian.preprocessing import remove_brightspot, gradient_norm, denoise, remove_background, stripe_correction
from minian.motion_correction import estimate_shift_fft, apply_shifts, interpolate_frame, mask_shifts
from minian.initialization import seeds_init, gmm_refine, pnr_refine, intensity_refine, ks_refine, seeds_merge, initialize
from minian.cnmf import get_noise_fft, get_noise_welch, update_spatial, update_temporal, unit_merge
from minian.visualization import VArrayViewer, MCViewer, CNMFViewer, generate_videos, visualize_temporal_update, normalize

### module initialization

normalize `dpath` and initialize package for visualization. Re-run this cell if you are having trouble getting the visualization to show up on the notebook.

In [ ]:
dpath = os.path.abspath(dpath)
hv.notebook_extension('bokeh', width=100)

## Pre-processing

### loading videos and visualization

The first argument of `load_videos` should be the path that contains the videos, which could be `dpath` we already defined. The second argument `pattern` is optional and is the [regular expression](https://docs.python.org/3/library/re.html) used to filter files under the specified folder. The default value **'msCam[0-9]+\.avi$'** means that a file can only be loaded if its filename contain **'msCam'** then followed by at least one digit of number then **'.avi'** as the end of the filename. This can be changed to suit the naming convention of your videos. `in_memory` specify whether the whole video should be loaded into memory. Currently it is recommended to set `in_memory` to `True`. `dtype` is the underlying [data types](https://docs.scipy.org/doc/numpy-1.15.0/user/basics.types.html) of the data. Usually `float32` is good enough and should be preferred to save memory demand. The resulting "video array" `varr` contains three dimensions: `height`, `width`, and `frame`. If you wish to downsample the video, pass in a `dictionary` to `resample`, whose keys should be the name of dimensions and values an integer specifying how many times that dimension should be reduced. For example, `resample={'frame': 2}` will temporally downsample the video with a factor of 2.

In [ ]:
%%time
varr = load_videos(dpath, in_memory=in_memory, dtype=np.float32, resample=None)

The previous cell load videos and concatenate them together into `varr`, which is a [xarray.DataArray](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html#xarray.DataArray). Now it's a perfect time to get familiar with this data structure and the [xarray](https://xarray.pydata.org/en/stable/) module in general, since we will be using these data structures throughout analysis. Basicly a `xarray.DataArray` is a labeled N-dimensional array. We can ask the computer to print out some information of `varr` by calling its name (as with any other variable):

In [ ]:
varr

We can see now that `varr` is a `xarray.DataArray` with a [name](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.name.html#xarray.DataArray.name) `'demo_movies'` and three dimensions: `frame`, `height` and `width`, each dimension is simply labeled with ascending natural numbers. The [dtype](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.dtype.html#xarray.DataArray.dtype) ([data type](https://docs.scipy.org/doc/numpy-1.14.0/user/basics.types.html)) of `varr` is `numpy.float32`

In addition to these information, we can visualize `varr` with the help of `VArrayViewer`, which shows the array as movie and also plot max, mean and minimum fluorescence:

In [ ]:
%%output size=100
vaviewer = VArrayViewer([varr], framerate=5)
display(vaviewer.widgets)
vaviewer.show()

### subset part of video

Before proceeding to pre-processing, it's good practice to check if there is something obviously wrong with the video (like camera suddenly drops dark). This can usually be observed by visualizing the video and check the mean fluorescence plot. To take out some bad `frame`, let's say, `frame` after 800, we can utilize the [xarray.DataArray.sel](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.sel.html) method and [slice](https://docs.python.org/3/library/functions.html#slice):

In [ ]:
varr_ref = varr.sel(frame=slice(None, 800))

This will subset `varr` along the `frame` dimension from the begining to the `frame` labeled **800**, and assign the result back to `varr_ref`, which is equivalent to taking out `frame` from **801** to the end. Note you can do the same thing to other dimensions like `height` and `width` to take out certain pixels of your video for all the `frame`s. For more information on using `xarray.DataArray.sel` as well as other indexing strategies, see [xarray documentation](http://xarray.pydata.org/en/stable/indexing.html)

If your `varr` is fine, just assign it to `varr_ref` to keep the naming consitent with later codes.

In [ ]:
varr_ref = varr

Now we chunk `varr` into smaller pieces to use parallelization to accelerate performance.

In [ ]:
varr_ref = varr_ref.chunk(dict(height=-1, width=-1, frame='auto'))

### bright spots removal

`remove_brightspot` tries to correct for brightspots that are probably caused by dust on the imaging sensor. Usually this type of artifacts occupies one pixel which is significantly brighter than its surrounding pixels. Thus, for any given pixel, `remove_brightspot` calculate a difference of intensity between that pixel and the mean of the **four** surrounding pixels (that is, four pixels that are connected to the pixel in question by edge). Then for each frame, this difference value is taken for all pixels, and a zscore of them is calculated. The pixels whose zscore of difference that exceed `thres` is treated as "brightspots", and whose value will be substituted by the mean of its four surrounding pixels. Thus `thres` controls how strict brightspot detection would be. In stead of passing in numbers, you can also pass a string `'min'`, which means use the inverse of the minimum (most negative) value of zscore as the threshold. If you believe your videos are free of this type of artifacts, feel free to skip this step to reduce artificial assumptions and biases.

For the following few cells during pre-processing, the functions (such as `remove_brightspot`) are evaluated lazily, which means it only creates a "plan" for the comuptation without doing it. Actual computations are carried out when we explicitly call the [persist](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.persist.html) method of the resulting `DataArray` (like `varr_ref.persist()`). This opens up possibility for [out-of-core computations](https://en.wikipedia.org/wiki/External_memory_algorithm), but it's currently not tested yet. So for now it is recommended to leave everything under the `if in_memory:` clause intact.

In [ ]:
%%time
varr_ref = remove_brightspot(varr_ref, thres=2)
if in_memory:
    with ProgressBar():
        varr_ref = varr_ref.persist()

### stripe correction

The earlier versions of miniscopes suffer from artifacts that looks like horizontal or vertical stripes across the image that's stationary across frames. `stripe_correction` tries to correct for this type of artifacts. Technically it calculate the mean along `'frame'` dimension (a "mean frame" if you will), then again calculate a mean along `reduce_dim` to get a one-dimensional representation of the "stripes", and then every frame in the movie is subtracted by this 1d stripe uniformly. Thus, `reduce_dim` should be the dimension (direction) the stripes run along. If you believe your videos are free from the stripes artifacts, feel free to skip this step to reduce artificial assumptions and biases.

In [ ]:
%%time
varr_ref = stripe_correction(varr_ref, reduce_dim='height')
if in_memory:
    with ProgressBar():
        varr_ref = varr_ref.persist()

### normalize

`scale_varr` is a convenient function that act on `DataArray`s to linearly rescale every value to the range of 0 to 1 inclusive. If you don't like it, pass in `scale` to change the range or skip it altogether. Note that `scale` takes a `tuple` of 2 elements representing lower and upper bound of the range.

In [ ]:
%%time
varr_ref = scale_varr(varr_ref, scale=(0, 1))
if in_memory:
    with ProgressBar():
        varr_ref = varr_ref.persist()

### save processed movie

The `save_variable` function takes in three required arguments: `var` is the variable you want to save, `fpath` is the path under which the actual data file will be stored, and `fname` is the file name of the data file. `meta_dict` is optional and will be used to fill in metadatas.

In particular, here we are saving our minimally-processed videos in `DataArray` format. We give it a "name" `"org"` by calling the [rename](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.rename.html) method on the array, which is `xarray`'s internal naming system that stays with the actual data and will be displayed when you print out the `DataArray` (In contrast, you can bind arbitrary variable name to the data like `whatever = varr_ref` without touching the data, which is less reliable and not accessible to other functions, which is probably why we need another name). In practice, give it a name that's human-readable and be sure to not name two pieces of data with the same name (Otherwise an error will occur if you try to combine them in a single dataset). In any case, the saved original videos are usually not used in the later part of the pipeline, except for the final `generate_videos` method which generate a movie to visualize the result of the pipeline. Thus if you are sure that you don't care about that movie, feel free to skip this to save both time and disk space.

In [ ]:
%%time
save_variable(var=varr_ref.rename("org"), fpath=dpath, fname='minian', meta_dict=meta_dict)

### estimate gradient

In [ ]:
%%time
with ProgressBar():
    varr_gradient = gradient_norm(varr_ref.isel(frame=0)).compute()

In [ ]:
kappa = varr_gradient.quantile(0.9).values

### anisotropic diffusion

In [ ]:
%%time
Y = denoise(varr_ref, 'anisotropic', niter=10, kappa=kappa, gamma=0.25, option=2)
if in_memory:
    with ProgressBar(), dask.config.set(scheduler='processes'):
        Y = Y.persist()

### background removal

In [ ]:
%%time
Y = remove_background(Y, method='tophat', wnd=10)
if in_memory:
    with ProgressBar(), dask.config.set(scheduler='processes'):
        Y = Y.persist()

### normalization

In [ ]:
%%time
Y = scale_varr(Y)
if in_memory:
    with ProgressBar():
        Y = Y.persist()

### visualization of pre-processing

In [ ]:
%%output size=70
vaviewer = VArrayViewer([varr_ref, Y.rename('Y')], framerate=5)
display(vaviewer.widgets)
vaviewer.show()

## motion correction

### prepare movie for motion correction

#### use old method to generate movie for motion correction

In [ ]:
%%time
varr_mc = remove_background(varr_ref, 'uniform', 51)
varr_mc = denoise(varr_mc, 'gaussian', ksize=(3, 3), sigmaX=0)
if in_memory:
    with ProgressBar():
        varr_mc = varr_mc.compute()

#### use Y for motion correction

In [ ]:
varr_mc = Y

### estimate shifts

In [ ]:
%%time
res = estimate_shift_fft(varr_mc, on='mean', pct_thres=99.9)
if in_memory:
    with ProgressBar():
        res = res.compute()
shifts = res.sel(variable = ['height', 'width'])
corr = res.sel(variable='corr')

### masking and interpolation

In [ ]:
%%time
shifts_ma, mask = mask_shifts(varr_fft, corr, shifts, z_thres=-1.5)

In [ ]:
%%time
varr_mc = interpolate_frame(varr_mc.compute().rename('varr_mc'), mask)

### determine shifts

#### take cumulative sum if `on='perframe'` when estimating shifts

In [ ]:
%%time
shifts_final = shifts.cumsum('frame')
shifts_final = np.around(shifts_final.fillna(0)).astype(int)

#### use raw shifts otherwise

In [ ]:
shifts_final = np.around(shifts.fillna(0)).astype(int)

### apply shifts

In [ ]:
Y_mc = apply_shifts(Y, shifts_final)
if in_memory:
    with ProgressBar():
        Y_mc = Y_mc.persist()

### visualization of shifts

In [ ]:
%%output size=100
%%opts Curve [width=500, tools=['hover']]
hv.NdOverlay(dict(width=hv.Curve(shifts.sel(variable='width')), height=hv.Curve(shifts.sel(variable='height'))))\
+ hv.NdOverlay(dict(width=hv.Curve(shifts_final.sel(variable='width')), height=hv.Curve(shifts_final.sel(variable='height'))))

### visualization of motion-correction

In [ ]:
%%output size=100 fps=5
%%opts Image (cmap='Viridis')
vaviewer = VArrayViewer([Y.rename('Y'), Y_mc.rename('Y_mc')], framerate=5)
display(vaviewer.widgets)
vaviewer.show().redim.range(Y=(0,0.2), Y_mc=(0, 0.2))

### save result as DataSet

In [ ]:
%%time
save_variable(Y_mc.rename('Y'), dpath, 'minian', meta_dict=meta_dict)

## initialization

In [ ]:
%%time
minian = xr.open_dataset(os.path.join(dpath, 'minian.nc'), autoclose=True)
Y = minian['Y'].load()

In [ ]:
%%time
seeds = seeds_init(Y, method='rolling')

In [ ]:
%%time
seeds_gmm = gmm_refine(Y, seeds)

In [ ]:
%%time
seeds_pnr = pnr_refine(Y, seeds_gmm)

In [ ]:
%%time
seeds_int = intensity_refine(Y, seeds_pnr)

In [ ]:
%%time
seeds_ks = ks_refine(Y, seeds_int)

In [ ]:
%%time
seeds_mrg = seeds_merge(Y, seeds_ks)

In [ ]:
%%time
A, C, b, f = initialize(Y, seeds_mrg, chk=dict(height=200, width=200, frame=1000))

In [ ]:
opts = dict(plot=dict(height=300, width=300))
regrid(hv.Image(A.sum('unit_id'), kdims=['width', 'height'])).opts(**opts) + regrid(hv.Image(C, kdims=['frame', 'unit_id'])).opts(**opts)

In [ ]:
%%time
save_variable(A.rename('A_init').rename(unit_id='unit_id_init'), dpath, 'minian', meta_dict=meta_dict)
save_variable(C.rename('C_init').rename(unit_id='unit_id_init'), dpath, 'minian', meta_dict=meta_dict)
save_variable(b.rename('b_init'), dpath, 'minian', meta_dict=meta_dict)
save_variable(f.rename('f_init'), dpath, 'minian', meta_dict=meta_dict)

## CNMF

### loading data

In [ ]:
%%time
chk = chunks.copy()
chk['unit_id_init'] = chk.pop('unit_id')
minian = xr.open_dataset(os.path.join(dpath, 'minian.nc'), chunks=chk)
Y = minian['Y']
A_init = minian['A_init'].rename(unit_id_init='unit_id')
C_init = minian['C_init'].rename(unit_id_init='unit_id')
b_init = minian['b_init']
f_init = minian['f_init']

### estimate spatial noise

In [ ]:
%%time
sn_spatial, psd = get_noise_fft(Y)

In [ ]:
%%opts Image [height=300, width=800, colorbar=True] (cmap='Viridis')
psd_flt = psd.stack(spatial=['height', 'width'])
hv_psd = hv.Image(psd_flt.assign_coords(spatial=range(psd_flt.sizes['spatial'])).rename('psd'), kdims=['spatial', 'freq'])
regrid(hv_psd).redim.range(psd=(0, 1e-4))

### randomly select units for parameter exploring

In [ ]:
units = np.random.choice(A_init.coords['unit_id'], 10)

### test parameters for spatial update

In [ ]:
opts_A = dict(plot=dict(height=480, width=752), style=dict(cmap='Viridis'))
opts_C = dict(plot=dict(height=480, width=1000), style=dict(cmap='Viridis'))
sprs_ls = [5e-6, 5e-3, 0.5, 5]
A_dict = dict()
for cur_sprs in sprs_ls:
    cur_A, cur_b, cur_C, cur_f = update_spatial(
        Y, A_init.sel(unit_id=units),
        b_init, C_init.sel(unit_id=units), f_init, sn_spatial, sparse_penal=cur_sprs)
    hv_cur_A = hv.Image(cur_A.sum('unit_id'), kdims=['width', 'height']).opts(**opts_A)
    hv_cur_C = hv.Image(cur_C, kdims=['frame', 'unit_id']).opts(**opts_C)
    A_dict[cur_sprs] = (hv_cur_A + hv_cur_C).cols(1)
hv_res = hv.HoloMap(A_dict, kdims=['sparse_penalty'])

In [ ]:
%%opts Image [colorbar=True] {+axiswise}
hv_res.collate()

### first spatial update

In [ ]:
%%time
A_spatial, b_spatial, C_spatial, f_spatial = update_spatial(
    Y, A_init, b_init, C_init, f_init, sn_spatial, sparse_penal=0.1)

In [ ]:
%%opts Image [colorbar=True] (cmap='Viridis')
regrid(hv.Image(A_init.sum('unit_id'), kdims=['width', 'height'])).opts(plot=dict(height=480, width=752))\
+ regrid(hv.Image(A_spatial.sum('unit_id').rename('A_spatial'), kdims=['width', 'height'])).opts(plot=dict(height=480, width=752)).redim.range(A_spatial=(0, 0.5))

### test parameters for temporal update

In [ ]:
%%time
import itertools as itt
p_ls = [1]
sprs_ls = [0.5, 5]
add_ls = [5]
noise_ls = [0.25]
vis_dict = dict()
for cur_sprs, cur_p, cur_add, cur_noise in itt.product(sprs_ls, p_ls, add_ls, noise_ls):
    print("processing {}".format((cur_p, cur_sprs, cur_add, cur_noise)))
    YrA, cur_C, cur_S, cur_B, cur_C0, cur_sig, cur_g, = update_temporal(
        Y, A_spatial.isel(unit_id=slice(10, 20)), b_spatial, C_spatial.isel(unit_id=slice(10, 20)),
        f_spatial, sn_spatial, sparse_penal=cur_sprs, p=cur_p, use_spatial=False, use_smooth=True,
        add_lag = cur_add, noise_freq=cur_noise, chk=dict(frame=200, unit_id=20),
        cvx_sched="processes")
    # YrA = YrA.assign_coords(unit_id = range(YrA.sizes['unit_id']))
    # cur_C = cur_C.assign_coords(unit_id = range(cur_C.sizes['unit_id']))
    # cur_S = cur_S.assign_coords(unit_id = range(cur_S.sizes['unit_id']))
    # cur_g = cur_g.assign_coords(unit_id = range(cur_g.sizes['unit_id']))
    # cur_sig = cur_sig.assign_coords(unit_id = range(cur_sig.sizes['unit_id']))
    vis_dict[(cur_p, cur_sprs, cur_add, cur_noise)] = visualize_temporal_update(
        YrA, cur_C, cur_S, cur_g, cur_sig)

In [ ]:
%%opts Curve [width=800] {+framewise}
hv_res = hv.HoloMap(vis_dict, kdims=['p', 'sparse_penalty', 'add_lag', 'noise_freq']).collate()
hv_res

### first temporal update

In [ ]:
%%time
YrA, C_temporal, S_temporal, B_temporal, C0_temporal, sig_temporal, g_temporal = update_temporal(
        Y, A_spatial,
        b_spatial, C_spatial, f_spatial, sn_spatial, jac_thres=0.2,
        noise_freq=0.45, sparse_penal=0.5, p=1, add_lag = 0, use_spatial=False, chk=dict(frame=2000, unit_id=200))

In [ ]:
%%opts Curve [width=1200] {+framewise}
visualize_temporal_update(YrA, C_temporal, S_temporal, g_temporal, sig_temporal, normalize=False).select(unit_id = slice(0, 50))

In [ ]:
%%opts Image [colorbar=True] (cmap='Viridis')
hv_c = regrid(hv.Image(C_temporal.rename('c'), kdims=['frame', 'unit_id'])).opts(plot=dict(height=500, width=1000)).redim.range(c=(0, 1))
hv_s = regrid(hv.Image(S_temporal.rename('s'), kdims=['frame', 'unit_id'])).opts(plot=dict(height=500, width=1000)).redim.range(s=(0, 0.006))
(hv_c + hv_s).cols(1)

### merge units

In [ ]:
%%time
A_mrg, sig_mrg = unit_merge(A_spatial, sig_temporal, thres_corr=0.8)

### second spatial update

In [ ]:
%%time
A_spatial_it2, b_spatial_it2, C_spatial_it2, f_spatial_it2 = update_spatial(
    Y, A_mrg, b_spatial, sig_mrg, f_spatial, sn_spatial, sparse_penal=0.1, dl_wnd=5)

In [ ]:
A_spatial_it2_norm = xr.apply_ufunc(normalize, A_spatial_it2, input_core_dims=[['height', 'width']], output_core_dims=[['height', 'width']], vectorize=True)

In [ ]:
regrid(hv.Image(A_spatial.sum('unit_id'), kdims=['width', 'height'])).opts(plot=dict(height=480, width=752))\
+ regrid(hv.Image(A_spatial_it2_norm.sum('unit_id'), kdims=['width', 'height'])).opts(plot=dict(height=480, width=752))

### second temporal update

In [ ]:
%%time
YrA, C_temporal_it2, S_temporal_it2, B_temporal_it2, C0_temporal_it2, sig_temporal_it2, g_temporal_it2 = update_temporal(
    Y, A_spatial_it2, b_spatial_it2, C_spatial_it2, f_spatial_it2, sn_spatial, jac_thres=0.2,
    noise_freq=0.45, sparse_penal=0.5, p=1, add_lag=0, chk=dict(frame=2000, unit_id=200))

In [ ]:
%%opts Curve [width=1200] {+framewise}
visualize_temporal_update(
    YrA, C_temporal_it2, S_temporal_it2, g_temporal_it2, sig_temporal_it2).select(unit_id=slice(0, 50))

In [ ]:
%%opts Image [colorbar=True, tools=['hover']] (cmap='Viridis')
hv_c = regrid(hv.Image(C_temporal_it2.rename('c'), kdims=['frame', 'unit_id'])).opts(plot=dict(height=500, width=1000)).redim.range(c=(0, 1))
hv_s = regrid(hv.Image(S_temporal_it2.rename('s'), kdims=['frame', 'unit_id'])).opts(plot=dict(height=500, width=1000)).redim.range(s=(0, 0.006))
(hv_c + hv_s).cols(1)

### save results

In [ ]:
%%time
minian.close()
save_variable(A_spatial_it2.rename('A'), dpath, 'minian', meta_dict=meta_dict)
save_variable(C_temporal_it2.rename('C'), dpath, 'minian', meta_dict=meta_dict)
save_variable(S_temporal_it2.rename('S'), dpath, 'minian', meta_dict=meta_dict)
save_variable(g_temporal_it2.rename('g'), dpath, 'minian', meta_dict=meta_dict)
save_variable(b_spatial_it2.rename('b'), dpath, 'minian', meta_dict=meta_dict)
save_variable(f_spatial_it2.rename('f'), dpath, 'minian', meta_dict=meta_dict)

### visualization

In [ ]:
minian = xr.open_dataset(os.path.join(dpath, 'minian.nc'))

In [ ]:
%%time
generate_videos(minian, os.path.join(dpath, "minian.mp4"), chk=dict(height=100, width=100, frame=1000))

In [ ]:
cnmfviewer = CNMFViewer(minian, minian['Y'])

In [ ]:
cnmfviewer.show()